# 对话机器人

## 单轮对话
<div>
<img src="figs/10_single_turn.jpg" width="1000"/>
</div>



## 多轮对话

<div>
<img src="figs/10_multi_turn_1.jpg" width="1000"/>
</div>

* 效率比较低下比如一个三轮的对话样本(Q_1,A_1,Q_2,A_2,Q_3,A_3) 需要拆分成三条数据
* (Q_1,A_1) -> (Q_1A_1Q_2, A_2) -> (Q_1A_1Q_2A_2Q_3, A_3)

<div>
<img src="figs/10_multi_turn_2.jpg" width="1000"/>
</div>

* 不拆分多轮对话，直接当成一轮。
* 上三角mask矩阵保证上一轮对话看不到下面的，但是之后的对话，可以看到上一轮的。

# 1. 导包

In [1]:
import os
# os.environ["WORLD_SIZE"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
# we do not use the DataCollatorForLanguageModeling because we need to pass the labels manually

# 2. 导入数据

In [2]:
ds = Dataset.load_from_disk('data/alpaca_data_zh')

In [3]:
ds[:3]

{'output': ['以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。',
  '4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。'],
 'input': ['', '输入：4/16', ''],
 'instruction': ['保持健康的三个提示。', '解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？']}

In [4]:
datasets = ds.train_test_split(test_size=0.2)

# 3. 预处理

In [5]:
ckpt = 'Langboat/bloom-389m-zh'
tokenizer = AutoTokenizer.from_pretrained(ckpt)

def process_function(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = example['instruction']
    input_str = example['input']
    instruction_input_seq = "\n".join(["Human: " + instruction, input_str]).strip() + "\n\n Assistant:"
    response_str = example['output'] + tokenizer.eos_token
    tokenized_instruction_input = tokenizer(instruction_input_seq)
    tokenized_response = tokenizer(response_str)
    input_ids = tokenized_instruction_input['input_ids'] + tokenized_response['input_ids']
    attention_mask = tokenized_instruction_input['attention_mask'] + tokenized_response['attention_mask']
    labels = [-100] * len(tokenized_instruction_input['input_ids']) + tokenized_response['input_ids']
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}
    

```
不管截断还是不截断，计算loss的方式还是不变的，参考因果语言模型的loss计算方式，它只会预测下一个token应该是什么，
如果最后token的下一个token就不是eos_token，那完全没影响，只是说这条回复没有训练完整，但是在其他的数据上还是会学习到什么时候要结束，
即解码出eos_token。当然了，肯定还是不截断最好。
```

In [6]:
tokenized_ds = datasets.map(process_function, remove_columns=ds.column_names) 

Map:   0%|          | 0/21486 [00:00<?, ? examples/s]

Map:   0%|          | 0/5372 [00:00<?, ? examples/s]

In [7]:
print(tokenized_ds['train'][13]['input_ids'][:10])
print(tokenized_ds['train'][13]['labels'][:10])

[23069, 29, 210, 21198, 3384, 1205, 17578, 1806, 671, 189]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


In [8]:
tokenizer.decode(tokenized_ds['train'][13]['input_ids'])

'Human: 列出人口前十大国家。\n\n Assistant:根据联合国2021年统计数据，世界人口前十大国家为（人口单位：亿）：\n1. 中国：13.98\n2. 印度：13.62\n3. 美国：3.311\n4. 印度尼西亚：2.734\n5. 巴基斯坦：2.221\n6. 巴西：2.139 \n7. 奈及利亚：2.078\n8. 孟加拉国：1.646\n9. 俄罗斯：1.443\n10. 墨西哥：1.297</s>'

In [9]:
tokenizer.decode(list(filter(lambda x: x!=-100, tokenized_ds['train'][13]['labels'])))

'根据联合国2021年统计数据，世界人口前十大国家为（人口单位：亿）：\n1. 中国：13.98\n2. 印度：13.62\n3. 美国：3.311\n4. 印度尼西亚：2.734\n5. 巴基斯坦：2.221\n6. 巴西：2.139 \n7. 奈及利亚：2.078\n8. 孟加拉国：1.646\n9. 俄罗斯：1.443\n10. 墨西哥：1.297</s>'

# 4. 创建模型

In [10]:
model = AutoModelForCausalLM.from_pretrained(ckpt)

args = TrainingArguments(   output_dir='./chatbot/',
                            num_train_epochs=2,
                            per_device_train_batch_size=8,
                            gradient_accumulation_steps=4,
                            per_device_eval_batch_size=8,
                            logging_steps=10,
                            load_best_model_at_end=True,
                            evaluation_strategy='epoch',
                            save_strategy='epoch',
                            save_total_limit=1,
                            report_to='wandb',
                        )
trainer = Trainer(model=model, args=args, train_dataset=tokenized_ds['train'], eval_dataset=tokenized_ds['test'],
                data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))

# 5. 训练

In [11]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yutongdai. Use `wandb login --relogin` to force relogin


You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=336, training_loss=2.1480806072553, metrics={'train_runtime': 1481.5528, 'train_samples_per_second': 29.005, 'train_steps_per_second': 0.227, 'total_flos': 1.2481074572722176e+16, 'train_loss': 2.1480806072553, 'epoch': 2.0})

# 6. 推理

In [22]:
from transformers import pipeline
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)
ipt = "Human: {}\n{}".format('考试有什么技巧', '').strip() + "\n\nAssistant: "
result = pipe(ipt, max_length=256)
print(result[0]['generated_text'])

Human: 考试有什么技巧

Assistant: 考试技巧包括许多方面，其中一项重要的技巧是作弊。作弊是一种常见的作弊手段，它旨在通过人为地更改考试内容来逃避考试。作弊不仅会影响考试的公正性和公平性，还会给考试组织者和参与者带来经济损失。因此，作弊是考试中不可忽视的一部分，需要广大考生和社会各界的共同努力来防止。


常用推理参数
* 长度控制
  * min/max_new_tokens: 最小/最大生成的长度
  * min/max_length: 序列整体的最小/最大长度
* 解码策略
  * do_sample: 是否启用采样的生成方式
  * num_beams: beam search的大小
* 采样参数
  * temperature: 默认1.0，即原始分布，低于1.0会使得分布更尖锁，高于1.0会使得分布更均匀
  * top_k:将词概率从大到小排列，将采样限制在前K个词
  * top_p:将词概率从大到小排列，将采样限制在前N个词，条件是这N个词的概率超过top_p的值
* 惩罚项
  * repetition_ penalty 重复惩罚项，实现原理是降低已经出现过的token的概率

In [27]:
result = pipe(ipt, max_length=256, num_beams=5)
print(result[0]['generated_text'])

Human: 考试有什么技巧

Assistant: 考试有很多技巧。以下是一些常见的技巧：

1. 预习和复习：预习和复习是考试中非常重要的一环。通过预习和复习，可以更好地理解考试内容，提高应试能力。

2. 准备充分：准备充分是考试中一个非常重要的步骤。在考试前进行充分的准备，可以确保考试的顺利进行。

3. 时间管理：时间管理是考试中一个非常重要的技能。在考试前制定一个计划，合理安排时间，可以有效提高考试效率。

4. 心理准备：心理准备是考试中一个非常重要的技能。在考试前进行充分的心理准备，可以有效降低考试焦虑，提高应试能力。

5. 注意细节：注意细节是考试中一个非常重要的技能。在考试前仔细检查每一道题的细节，可以有效提高应试能力。

6. 注意语法和拼写：注意语法和拼写是考试中一个非常重要的技能。在考试前进行充分的语法和拼写准备，可以有效提高应试能力。

7. 复习巩固：复习巩固是考试中一个非常重要的技能。在考试前进行充分的复习，可以有效巩固考试


In [28]:
result = pipe(ipt, max_length=256, do_sample=True)
print(result[0]['generated_text'])

Human: 考试有什么技巧

Assistant: 考试技巧是指解决考试中出现的问题及解决考试中的干扰因素的方法。在准备考试时，你可以采取以下几点应对技巧：

1. 根据题目的特点，选择适当的解题技巧。

2. 使用正确的作弊器材。

3. 上手练习，循序渐进进行解答。

4. 为避免过度紧张而放弃一些基础的技巧。

5. 认真审题，仔细思考每一题的背景信息。

6. 注意细节问题，防止漏掉不必要的细节。

7. 注意作弊行为的纠正。

请注意，这不是一个全面且成功的应对措施，不同的考试需要不同的技巧。但是，这些建议可以帮助你提高你的能力，应对考试中的不确定的因素。


In [29]:
ipt = "Human: {}\n{}".format('给出做黄焖鸡的步骤', '').strip() + "\n\nAssistant: "
result = pipe(ipt, max_length=512)
print(result[0]['generated_text'])

Human: 给出做黄焖鸡的步骤

Assistant: 准备食材：
1. 鸡胸肉：鸡胸肉去皮切块。
2. 鸡肉：将鸡肉剁成小块。
3. 姜：切片，切碎。
4. 蒜：切片，切碎。
5. 盐：适量，根据个人口味增减。
6. 橄榄油：适量，根据个人口味增减。
7. 鸡精：适量，根据个人口味增减。
8. 番茄酱：适量，根据个人口味增减。
9. 橄榄油：适量，根据个人口味增减。
10. 番茄：切碎，撒上盐。
11. 番茄酱：适量，根据个人口味增减。
12. 鸡肉块：将鸡肉块剁成小块。
13. 番茄酱：适量，根据个人口味增减。
14. 橄榄油：适量，根据个人口味增减。
15. 番茄：切碎，撒上盐。
16. 鸡肉块：将鸡肉块剁成小块。
17. 番茄酱：适量，根据个人口味增减。
18. 橄榄油：适量，根据个人口味增减。
19. 番茄：切碎，撒上盐。
20. 鸡肉块：将鸡肉块剁成小块。


In [34]:
ipt = "Human: {}\n{}".format('给出做黄焖鸡的步骤', '').strip() + "\n\nAssistant: "
result = pipe(ipt, max_length=512, repetition_penalty=1.2, num_beams=5)
print(result[0]['generated_text'])

/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Human: 给出做黄焖鸡的步骤

Assistant: 制作黄焖鸡的步骤如下：

1. 准备食材：鸡胸肉，鸡肉，大蒜，姜，盐，胡椒粉，料酒。
2. 腌制鸡肉：将鸡肉剁成小块，放入沸水中，加入盐，胡椒粉和料酒。
3. 焯水：将鸡肉放入沸水中，加入适量的水，煮至断生。
4. 捞出鸡肉：将鸡肉捞出，沥干水分。
5. 煎炸：将鸡肉放入锅中，加入适量的油，炸至金黄色。
6. 捞出鸡肉：将鸡肉捞出，沥干油。
7. 切片：将鸡肉切成薄片，放入盘中。
8. 淋上酱汁：将鸡肉淋上酱汁，撒上蒜末和姜末。
9. 享用：将鸡胸肉和鸡肉放在盘子上，撒上蒜末和姜末，即可享用美味的黄焖鸡。


--- Logging error ---
Traceback (most recent call last):
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/logging/__init__.py", line 1104, in emit
    self.flush()
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/logging/__init__.py", line 1084, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/threading.py", line 973, in _bootstrap
    self._bootstrap_inner()
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/yutong/anaconda3/envs/IMP/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 279, in _process
 